# MLP Cifar 10

Hier werden das erste Mal im CIFAR10 nun Algorithmen aus dem Multilayer Perceptron (MLP) für multi-class Klassifikation verwendet:


In [1]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.models import model_from_json
from tensorflow.python.keras.datasets import cifar10
from pathlib import Path
import matplotlib.pyplot as plt

Using TensorFlow backend.
c:\users\1810837475\.conda\envs\testtensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\1810837475\.conda\envs\testtensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\1810837475\.conda\envs\testtensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\18108

In [2]:
batch_size = 128
num_classes = 10
epochs = 20

In [3]:
(train_X, train_y), (test_X, test_y) = cifar10.load_data()
# the data, split between train and test sets
print((train_X.shape))
print((train_y.shape))
print((test_X.shape))
print((test_y.shape))
print()
train_X = train_X.reshape(50000,32*32*3)
test_X = test_X.reshape(10000,32*32*3)
print((train_X.shape))
print((train_y.shape))
print((test_X.shape))
print((test_y.shape))

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)

(50000, 3072)
(50000, 1)
(10000, 3072)
(10000, 1)


In [4]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X /= 255 # skalieren
test_X /= 255 # Skalieren
print(train_X.shape[0], 'train samples')
print(test_X.shape[0], 'test samples')


train_y = keras.utils.to_categorical(train_y, num_classes)
test_y = keras.utils.to_categorical(test_y, num_classes)

50000 train samples
10000 test samples


In [5]:
# Prüfen ob bereits ein Model bereits trainiert und gespeichert ist, im Falle, dass noch kein Modell vorliegt neu erstellen
modelMLP = Path("./MODEL/modelMLP.json")
if modelMLP.is_file():
   # load json and create model
    json_file = open('./MODEL/modelMLP.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    modelMLP = model_from_json(loaded_model_json)
    # load weights into new model
    modelMLP.load_weights("./MODEL/modelMLP.h5")
    print("Loaded model from disk")
else:
    modelMLP = Sequential()
    modelMLP.add(Dense(512, activation='relu', input_shape=(3072,)))
    modelMLP.add(Dropout(0.2))
    modelMLP.add(Dense(512, activation='relu'))
    modelMLP.add(Dropout(0.2))
    modelMLP.add(Dense(num_classes, activation='softmax'))
    modelMLP.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
_________________________________________________________________


In [6]:
modelMLP.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [ ]:
history = modelMLP.fit(train_X, train_y,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(test_X, test_y))


Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 68s 1ms/step - loss: 2.1455 - accuracy: 0.2466 - val_loss: 1.8207 - val_accuracy: 0.3341
Epoch 2/20
50000/50000 [==============================] - 64s 1ms/step - loss: 1.8562 - accuracy: 0.3245 - val_loss: 1.8327 - val_accuracy: 0.3323
Epoch 3/20
50000/50000 [==============================] - 65s 1ms/step - loss: 1.7852 - accuracy: 0.3570 - val_loss: 1.7213 - val_accuracy: 0.3821
Epoch 4/20
50000/50000 [==============================] - 66s 1ms/step - loss: 1.7329 - accuracy: 0.3798 - val_loss: 1.6587 - val_accuracy: 0.4082
Epoch 5/20
50000/50000 [==============================] - 64s 1ms/step - loss: 1.6960 - accuracy: 0.3921 - val_loss: 1.5995 - val_accuracy: 0.4327
Epoch 6/20
50000/50000 [==============================] - 77s 2ms/step - loss: 1.6722 - accuracy: 0.4021 - val_loss: 1.6250 - val_accuracy: 0.4195
Epoch 7/20
50000/50000 [==============================] - 76s 2ms/s

In [ ]:
# Save trained Model to disk via JSON
model_json = modelMLP.to_json()
with open("./MODEL/modelMLP.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
modelMLP.save_weights("./MODEL/modelMLP.h5")
print("Saved modelMLP to disk")

In [ ]:
score = modelMLP.evaluate(test_X, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Plot training & validation accuracy values

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()